# Тетрадка

тетрадка для подведения итогов расчета

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

In [ ]:
calculated_wells =  preproc_tool.find_full_path_by_pattern(os.getcwd(), f"*_adapt_and_restore_metrics_report.xlsx*", 'restore')


In [ ]:
calculated_wells

In [ ]:
real_calculated_wells =[]
for i in calculated_wells:
    if len(i) <= len(calculated_wells[0]):
        real_calculated_wells.append(i)
    

In [ ]:
real_calculated_wells

In [ ]:
metrics = []
for this_path in real_calculated_wells:
    overall_metrics = pd.read_excel(this_path)
    
    well_name = this_path.split('\\')
    well_name = well_name[-1]
    well_name = well_name.split('_adapt_and_restore_metrics_')
    well_name = well_name[0]
    overall_metrics['№ скважины'] = [well_name]
    metrics.append(overall_metrics)

In [ ]:
border = datetime.datetime(2020,2,5)

In [ ]:
forecast = []
for this_path in real_calculated_wells:
    this_dict = {}
    
    well_name = this_path.split('\\')
    well_name = well_name[-1]
    well_name = well_name.split('_adapt_and_restore_metrics_')
    well_name = well_name[0]
    
    this_dict['Номер скважины'] = [well_name]
    
    this_forecast = pd.read_csv(this_path, index_col = 'Время', parse_dates = True)
    real_df= this_forecast[this_forecast.index < border]
    forecast_df = this_forecast[this_forecast.index >= border]
    this_dict['Дебит жидкости фактический на конец периода, м3/сут'] = real_df.tail(1)['Дебит жидкости, м3/сут'].values
    this_dict['Дебит жидкости предсказанный на конец периода, м3/сут'] = [forecast_df['Дебит жидкости, м3/сут (PREDICTION)'].dropna().values[0]]
    this_dict['Дебит жидкости линией тренда, м3/сут'] = forecast_df.head(1)['Дебит жидкости, м3/сут'].values
    this_dict['Относительное  изменение дебита жидкости по калибировкам, %'] = ((this_dict['Дебит жидкости предсказанный на конец периода, м3/сут'] - 
                                                                this_dict['Дебит жидкости фактический на конец периода, м3/сут'])
                                                               /this_dict['Дебит жидкости фактический на конец периода, м3/сут'] * 100)
    this_dict['Относительное  изменение дебита жидкости по линии тренда, %'] = ((this_dict['Дебит жидкости линией тренда, м3/сут'] - 
                                                                this_dict['Дебит жидкости фактический на конец периода, м3/сут'])
                                                               /this_dict['Дебит жидкости фактический на конец периода, м3/сут'] * 100)
 
    this_dict['Дата прогноза'] = [forecast_df.index[0]]
    this_df = pd.DataFrame(this_dict)
    forecast.append(this_df)
    
    

    

In [ ]:
result = metrics[0]
for i in metrics[1:]:
    result = result.append(i)
result

In [ ]:
import datetime
mark = str(datetime.datetime.today()).replace(':','_')

In [ ]:
result.to_excel(f'result_forecast_{mark}.xlsx')